# Práce s kamerou Basler v Jupyter Notebooku
Vzhledem k tomu, že software Pylon je nejen vlastní program pro připojení a manipulaci s kamerami, ale zároveň SDK, budeme používat přímo SDK. SDK je psáno v C, C++ nebo C#. Basler k tomu však v roce 2018 vytvořil wrapper do Pythonu s názvem `pypylon`.

My jsme k tomu vytvořili rozhraní s názvem `pypylon_opencv_viewer`. To umožňuje se připojit ke konkrétní kameře, zobrazit si snímky z kamery a dále nastavovat některé parametry kamery.

## Nastavení
Import potřebných knihoven

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import yaml
from pypylon import pylon 

from IPython.display import Image, display, clear_output

%matplotlib inline

Úvodni nastavení parametrů kamery. Je potřeba dopsat správné **sériové číslo** vlastní kamery. 

V případě, že se nastavuje cesta k ukládání fotek, je třeba nahradit veškerá `\` za `\\`. Při defaultním nastavení se fotky ukládají do složky s tímto ipynb.

In [ ]:
serial_number = '21991061' ### vlastní sériové číslo kamery
width = 1280
height = 1080
image_folder = '.' ### defaultní nastavení složky pro ukládání snímků

Připojení kamery pomocí `pypylonu`. Je zapotřebí zkontrolovat sériové číslo kamery.

In [ ]:
# Pypylon gets camera by serial number
info = None
for i in pylon.TlFactory.GetInstance().EnumerateDevices():
    if i.GetSerialNumber() == serial_number:
        info = i
        break
else:
    print('Camera with {} serial number not found'.format(serial_number))

# VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
if info is not None:
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
    camera.Open()

Pomocná funkce pro zobrazování obrázků

In [ ]:
def show_image(img):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, width, height)
    cv2.imshow(window_name, img)

Vytvoření GUI widgetů pro manipulaci s parametry kamery. Zásady pro tvorbu JSON notace widgetů viz [link](https://github.com/mbalatsko/pypylon-opencv-viewer#list-of-features).

In [ ]:
# List of features to create widgets
features = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 3500
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

Funkce pro image processing. Zatím pouze zobrazuje obrázky.

In [ ]:
def impro(img):
    show_image(img)
    return img

## Main
Výkonný kus kódu využívající vlastní knihovnu pro práci s kamerou a knihovnou opencv. Umožňuje nastavení parametrů pomocí GUI widgetů.
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(features)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot(window_size=(width, height), image_folder=image_folder)